## **Import Section**

In [ ]:
!pip install datasets
!pip install pyarrow==9.0.0
!pip install pydub
!pip install evaluate
!pip install jiwer
!pip install transformers[torch]
!pip install accelerate -U

  Using cached pyarrow-9.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.3 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.0.0
    Uninstalling pyarrow-16.0.0:
      Successfully uninstalled pyarrow-16.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.0 requires pyarrow>=12.0.0, but you have pyarrow 9.0.0 which is incompatible.
  Using cached pyarrow-16.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.8 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch
from huggingface_hub import HfApi, HfFolder
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, DatasetDict, Audio, Dataset, ClassLabel
from pydub import AudioSegment
import os
import io
import pandas as pd
import evaluate
import re
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
import numpy as np
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import csv
import pandas as pd
import jiwer
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import librosa

## **Drive Mount Section**

In [ ]:
# 1
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Functions Section**

In [ ]:

from nltk.tokenize import word_tokenize
import re
def tokenize_and_lowercase(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Iterate through each word
    for i in range(len(words)):
        word = words[i]

        # Check if the word contains English alphabets using regex
        if re.search('[a-zA-Z]', word):
            # Convert the word to lowercase
            words[i] = word.lower()

    # Return the modified sentence
    return ' '.join(words)


In [ ]:
import re

def remove_vowels(sentence):
    # Define a regular expression pattern to match Arabic vowels
    arabic_vowels_pattern = "[\u064B-\u0652\u0640]"  # Range of Arabic vowels

    # Use the re.sub() function to replace vowels with an empty string
    sentence_without_vowels = re.sub(arabic_vowels_pattern, '', sentence)

    return sentence_without_vowels

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
# data augmentation functions



# time stretch speed up or slow down
def time_stretch(signal, stretch_rate):
    return librosa.effects.time_stretch(signal, stretch_rate)

# pitch scaling
def pitch_scale(signal, sr, num_semitones):
    return librosa.effects.pitch_shift(signal, sr, num_semitones)

In [ ]:
def add_period(sentence):
    # Check if the sentence already ends with a period
    if sentence.strip()[-1] != '.':
        return sentence.strip() + '.'
    else:
        return sentence.strip()

## **RUN**

In [ ]:
# 2
data = pd.read_csv('test_data.csv')
x_test = data['old_index']
y_test = data['Sentence']

In [ ]:
# 3
x_test.iloc[0]

769

In [ ]:
# 4
data1 = pd.read_csv('train_data.csv')
x_train = data1['old_index']
y_train = data1['Sentence']

In [ ]:
# 5
x_train.iloc[0]

1500

In [ ]:
# Apply the function to each sentence in the 'train' DataFrame
y_train = y_train.apply(lambda x: add_period(x))
y_test = y_test.apply(lambda x: add_period(x))

In [ ]:
y_train

0                            قم بضبط المؤقت لمدة 9 دقائق.
1          يُرجى الانتظار 30 دقيقة ومن ثم قم بغلق الجهاز.
2         ارصد لي ملف السيناريوهات لمشروع التجديد الحضري.
3                    اعثر على ملف Quality Assurance Plan.
4                     فَتَّشَ في الحاسوب عن مجلد Results.
                              ...                        
1539    ابحث في المستندات عن ملف تقرير الاستجابة للعملاء.
1540    اكتب في ملحوظاتي ذهبت إلى المتجر اليوم لشراء ب...
1541     بعد ساعتين وربع قم بإغلاق برنامج Microsoft Word.
1542            عِد ساعة وربع ثم قم بإغلاق الجهاز مؤقتاً.
1543        اعثر في المستندات على ملف تقرير البحث الفلكي.
Name: Sentence, Length: 1544, dtype: object

## **Start**

In [ ]:
# 6
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Here's start the Fine_tuning process**


In [ ]:
len(x_train), len(x_test)

(1544, 387)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Initialize DatasetDict with empty keys
common_voice = DatasetDict({"train": None, "test": None})

# Initialize lists to store train and test data
train_data = {"audio": [], "sentence": []}
test_data = {"audio": [], "sentence": []}

# # Initialize counters for train and test data
# train_count = 0
# test_count = 0

def add_white_noise(signal, noise_factor):
    noise = np.random.normal(0, signal.std(), signal.size)
    augmented_signal = signal + noise * noise_factor
    return augmented_signal

AudioData = '/content/drive/MyDrive/all'
# augmented_samples = 0  # Counter for augmented samples


for filename, sentence in zip(x_train, y_train):
  file_path = os.path.join(AudioData, f'{filename}.mp3')
  audio = AudioSegment.from_mp3(file_path)
  new_sentence = remove_vowels(tokenize_and_lowercase(sentence))
  # Convert audio to bytes
  with io.BytesIO() as byte_io:
      audio.export(byte_io, format='wav')  # Export audio to WAV format and save to BytesIO object
      audio_bytes = byte_io.getvalue()
  train_data["audio"].append({"bytes": audio_bytes, "path": file_path})  # Provide bytes and file path
  train_data["sentence"].append(new_sentence)


for filename, sentence in zip(x_test, y_test):
  file_path = os.path.join(AudioData, f'{filename}.mp3')
  print(file_path)
  audio = AudioSegment.from_mp3(file_path)
  new_sentence = remove_vowels(tokenize_and_lowercase(sentence))
  # Convert audio to bytes
  with io.BytesIO() as byte_io:
      audio.export(byte_io, format='wav')  # Export audio to WAV format and save to BytesIO object
      audio_bytes = byte_io.getvalue()
  test_data["audio"].append({"bytes": audio_bytes, "path": file_path})  # Provide bytes and file path
  test_data["sentence"].append(new_sentence)

# Create Dataset objects
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

# Add datasets to DatasetDict
common_voice["train"] = train_dataset
common_voice["test"] = test_dataset

print('===================================================')
print(common_voice["train"][0])
print('===================================================')
print(common_voice["test"][0])
print('===================================================')

# Display the datatype of common_voice
print(len(common_voice["train"]))
print(len(common_voice["test"]))

/content/drive/MyDrive/all/769.mp3
/content/drive/MyDrive/all/1568.mp3
/content/drive/MyDrive/all/641.mp3
/content/drive/MyDrive/all/204.mp3
/content/drive/MyDrive/all/526.mp3
/content/drive/MyDrive/all/14.mp3
/content/drive/MyDrive/all/1685.mp3
/content/drive/MyDrive/all/580.mp3
/content/drive/MyDrive/all/1082.mp3
/content/drive/MyDrive/all/36.mp3
/content/drive/MyDrive/all/210.mp3
/content/drive/MyDrive/all/942.mp3
/content/drive/MyDrive/all/699.mp3
/content/drive/MyDrive/all/166.mp3
/content/drive/MyDrive/all/1698.mp3
/content/drive/MyDrive/all/1114.mp3
/content/drive/MyDrive/all/383.mp3
/content/drive/MyDrive/all/1164.mp3
/content/drive/MyDrive/all/850.mp3
/content/drive/MyDrive/all/154.mp3
/content/drive/MyDrive/all/54.mp3
/content/drive/MyDrive/all/1314.mp3
/content/drive/MyDrive/all/1548.mp3
/content/drive/MyDrive/all/249.mp3
/content/drive/MyDrive/all/1456.mp3
/content/drive/MyDrive/all/1850.mp3
/content/drive/MyDrive/all/1660.mp3
/content/drive/MyDrive/all/67.mp3
/content/driv

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# - Load Feature extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium")

# - Load Tokenizer : WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="Arabic", task="transcribe")

input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

# STEP 3. Combine elements with WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="Arabic", task="transcribe")

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print('===================================================')
print(common_voice["train"][0])
print(common_voice["test"][0])
print('===================================================')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'audio': {'path': '/content/drive/MyDrive/all/1500.mp3', 'array': array([-1.25420465e-05, -2.98434188e-05, -3.27390735e-05, ...,
       -3.78019293e-04, -3.40332685e-04,  0.00000000e+00]), 'sampling_rate': 16000}, 'sentence': 'قم بضبط المؤقت لمدة 9 دقائق .'}
{'audio': {'path': '/content/drive/MyDrive/all/769.mp3', 'array': array([-1.16529356e-12, -1.93138886e-12, -1.97343422e-12, ...,
       -1.80115975e-08,  1.56963544e-08, -1.57805200e-08]), 'sampling_rate': 16000}, 'sentence': 'جد لي ملف research data .'}


In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

Map:   0%|          | 0/1544 [00:00<?, ? examples/s]

Map:   0%|          | 0/387 [00:00<?, ? examples/s]

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
# load a pretrained checkpoint
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

model.generation_config.language = "arabic"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ar-en",  # change to a repo name of your choice
    per_device_train_batch_size=2,
    gradient_accumulation_steps=6,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=25,
    max_steps=200,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=100,
    save_steps=50,
    eval_steps=50,
    logging_steps=50,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


print("Training Started.")
trainer.train()
print("Training Ended.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

max_steps is given, it will override any value given in num_train_epochs


Training Started.


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
50,1.337000,0.303213,9.412105
100,0.112200,0.080741,6.429192
150,0.070900,0.067280,5.618303
200,0.029200,0.060497,4.691573


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.ut

Training Ended.


In [ ]:
model.save_pretrained("model")
tokenizer.save_pretrained("model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.json',
 'model/merges.txt',
 'model/normalizer.json',
 'model/added_tokens.json')

In [ ]:
import shutil

shutil.copytree('/content/model', '/content/drive/MyDrive/whisperModel')

'/content/drive/MyDrive/whisperModel'